Nhóm 18
- Đỗ Tấn Lực - 23520903
- Lê Quang Long - -23520878

#### Import các thư viện cần thiết cho xử lý dữ liệu, xây dựng mô hình, và Spark

- pandas, numpy: Xử lý dữ liệu.
- torch, torch.nn, torch.optim: Xây dựng và huấn luyện mô hình deep learning.
- sklearn: Tiền xử lý, chia tập, đánh giá mô hình.
- re: Xử lý chuỗi, regex.
- kagglehub: Tải dataset từ Kaggle.
- pyspark: Xử lý dữ liệu lớn với Spark, định nghĩa UDF.
- transformers: Sử dụng mô hình BERT (PhoBERT).

In [ ]:
import pandas as pd # Thư viện xử lý dữ liệu dạng bảng
import numpy as np # Thư viện xử lý mảng và các phép toán số học
import torch # Thư viện PyTorch cho deep learning
import torch.nn as nn # Các lớp mạng neural trong PyTorch
import torch.optim as optim # Các thuật toán tối ưu hóa trong PyTorch
from torch.utils.data import Dataset, DataLoader # Công cụ để làm việc với dữ liệu trong PyTorch
from sklearn.model_selection import train_test_split # Chia tập dữ liệu
from sklearn.preprocessing import LabelEncoder # Mã hóa nhãn
from sklearn.metrics import accuracy_score # Tính độ chính xác
from collections import Counter # Đếm tần suất
import re # Thư viện xử lý biểu thức chính quy (regex)
import kagglehub # Thư viện tải dataset từ Kaggle
from pyspark.sql import SparkSession # SparkSession để làm việc với Spark SQL
from pyspark.sql.types import DoubleType, ArrayType, StringType, IntegerType # Các kiểu dữ liệu của Spark
from pyspark.sql.functions import pandas_udf, PandasUDFType # Định nghĩa Pandas UDF trong Spark
from pyspark.ml.functions import predict_batch_udf # UDF cho dự đoán batch trong Spark ML (không dùng trong code này)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator # Đánh giá mô hình phân loại đa lớp trong Spark ML
from transformers import AutoTokenizer, AutoModel # Thư viện Hugging Face Transformers cho các mô hình Pre-trained (không dùng trong code này)

#### Tải dataset ABSA tiếng Việt từ Kaggle

- Sử dụng kagglehub để tải về bộ dữ liệu.
- Đọc các file train, test, val bằng pandas.

In [ ]:
# Định nghĩa đường dẫn đến dataset đã tải về
path = kagglehub.dataset_download("cthng123/absa-vietnamese")

# Đọc các file csv vào DataFrame của pandas
train = pd.read_csv(f"{path}/train_data.csv")
test = pd.read_csv(f"{path}/test_data.csv")
val = pd.read_csv(f"{path}/val_data.csv")

### Tiền xử lý dữ liệu văn bản

Ở bước này, chúng ta sẽ làm sạch dữ liệu review bằng cách loại bỏ emoji, ký tự đặc biệt, chuyển về chữ thường và tách từ. Đây là bước quan trọng giúp chuẩn hóa dữ liệu đầu vào cho mô hình học máy.

#### 1. Tạo regex để loại bỏ emoji và ký tự đặc biệt

Đoạn regex dưới đây giúp nhận diện và loại bỏ các emoji, biểu tượng, ký tự đặc biệt thường xuất hiện trong văn bản mạng xã hội, đánh giá sản phẩm, v.v.

**Tạo regex để loại bỏ emoji và ký tự đặc biệt:**

In [ ]:
# Định nghĩa biểu thức chính quy (regex) để loại bỏ emoji và các ký tự đặc biệt
emoji = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u'\U00010000-\U0010ffff'
    u"\u200d"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\u3030"
    u"\ufe0f"
"]+", flags=re.UNICODE)

## - Đoạn regex này giúp nhận diện và loại bỏ các emoji, biểu tượng, ký tự đặc biệt thường xuất hiện trong văn bản mạng xã hội, đánh giá sản phẩm, v.v.

#### 2. Định nghĩa hàm tiền xử lý văn bản

Hàm `preprocess` sẽ:
- Loại bỏ emoji bằng regex ở trên.
- Loại bỏ ký tự đặc biệt, chỉ giữ lại chữ cái, số và khoảng trắng.
- Chuyển toàn bộ về chữ thường.
- Chuẩn hóa khoảng trắng.
- Tách từ thành list (tokens).

In [ ]:
# Định nghĩa hàm tiền xử lý văn bản
def preprocess(text):
  # Kiểm tra nếu đầu vào không phải là chuỗi thì trả về nguyên giá trị
  if not isinstance(text, str):
    return text
  text = re.sub(emoji, '', text) # Loại bỏ emoji sử dụng regex đã định nghĩa
  text = re.sub(r"[^\w\s]", "", text)  # Loại bỏ ký tự đặc biệt, chỉ giữ lại chữ cái, số và khoảng trắng
  text = text.lower() # Chuyển toàn bộ văn bản về chữ thường
  text = re.sub(r"\s+", " ", text).strip() # Chuẩn hóa khoảng trắng (thay thế nhiều khoảng trắng bằng một khoảng trắng duy nhất và xóa khoảng trắng ở đầu cuối)
  return text.split() # Tách văn bản thành danh sách các từ (tokens)

#### 3. Áp dụng hàm tiền xử lý lên dữ liệu

Áp dụng hàm `preprocess` cho từng review trong tập train, test, val để thu được danh sách token.

In [ ]:
# Áp dụng hàm preprocess cho cột 'Review' trong các DataFrame
train['tokens'] = train['Review'].apply(preprocess)
test['tokens'] = test['Review'].apply(preprocess)
val['tokens'] = val['Review'].apply(preprocess)

# Gom tất cả token từ tập train lại để xây dựng từ điển
all_tokens = [token for tokens in train['tokens'] for token in tokens]
# Hiển thị danh sách tất cả các token
all_tokens

['giày',
 'đẹp',
 'đi',
 'êm',
 'lắm',
 'mình',
 'săn',
 'sale',
 'với',
 'giá',
 'khá',
 'rẻ',
 'chất',
 'lượng',
 'ok',
 'shipper',
 'thân',
 'thiện',
 'sẽ',
 'tiếp',
 'tục',
 'ủng',
 'hộ',
 'shop',
 'hình',
 'ảnh',
 'và',
 'video',
 'chỉ',
 'mang',
 'tính',
 'chất',
 'minh',
 'họa',
 'thôi',
 'về',
 'sản',
 'phẩm',
 'tính',
 'thang',
 'điểm',
 'thì',
 '9',
 '10',
 'ạ',
 'khá',
 'tốt',
 'hợp',
 'với',
 'giá',
 'tiền',
 'shipper',
 'thì',
 'ân',
 'cần',
 'thân',
 'thiện',
 'giao',
 'hàng',
 'khá',
 'nhanh',
 'ạ',
 'mình',
 'đặt',
 'size',
 '39',
 'nhưng',
 'chật',
 'k',
 'đeo',
 'nổi',
 'còn',
 'giầy',
 'thì',
 'đẹp',
 'giao',
 'hàng',
 'nhanh',
 'nên',
 'mua',
 'nha',
 'mọi',
 'người',
 'đẹp',
 'xuất',
 'sắc',
 'lun',
 'ạ',
 'giày',
 'đẹp',
 'giống',
 'hình',
 'mọi',
 'người',
 'nên',
 'mua',
 'tăng',
 '1',
 'size',
 'để',
 'khỏi',
 'phải',
 'mang',
 'chật',
 'chân',
 'đáng',
 'mua',
 'mọi',
 'người',
 'nên',
 'mua',
 'giao',
 'hàng',
 'nhanh',
 'đặt',
 'đế',
 'đen',
 'giao',
 'đế',


#### 4. Xây dựng từ điển (vocab) và ánh xạ từ sang chỉ số

- Đếm tần suất xuất hiện của từng token.
- Chỉ giữ lại các token xuất hiện nhiều hơn 1 lần, thêm token đặc biệt `<pad>`, `<unk>`.
- Tạo ánh xạ từ sang chỉ số (word_to_idx).

In [ ]:
# Đếm tần suất xuất hiện của từng token trong all_tokens
vocab = Counter(all_tokens)
# Xây dựng từ điển (vocab): thêm token đặc biệt <pad> (cho padding), <unk> (cho từ không biết)
# Chỉ giữ lại các token xuất hiện nhiều hơn 1 lần
vocab = ['<pad>', '<unk>'] + [word for word, fre in vocab.most_common() if fre > 1]
# Tạo ánh xạ từ từ (word) sang chỉ số (index)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
# Hiển thị ánh xạ từ điển
word_to_idx

Counter({'giày': 4699,
         'đẹp': 4045,
         'đi': 1838,
         'êm': 693,
         'lắm': 1532,
         'mình': 1294,
         'săn': 98,
         'sale': 106,
         'với': 1315,
         'giá': 1958,
         'khá': 682,
         'rẻ': 783,
         'chất': 1687,
         'lượng': 941,
         'ok': 1012,
         'shipper': 190,
         'thân': 225,
         'thiện': 196,
         'sẽ': 863,
         'tiếp': 290,
         'tục': 79,
         'ủng': 775,
         'hộ': 827,
         'shop': 2364,
         'hình': 980,
         'ảnh': 829,
         'và': 965,
         'video': 165,
         'chỉ': 608,
         'mang': 1278,
         'tính': 587,
         'minh': 165,
         'họa': 62,
         'thôi': 202,
         'về': 397,
         'sản': 921,
         'phẩm': 915,
         'thang': 9,
         'điểm': 136,
         'thì': 1221,
         '9': 11,
         '10': 121,
         'ạ': 978,
         'tốt': 642,
         'hợp': 623,
         'tiền': 845,
         'ân':

#### 5. Chuyển tokens thành chỉ số và padding

- Chuyển mỗi danh sách token thành danh sách chỉ số (index).
- Padding hoặc cắt bớt về độ dài cố định (MAX_LEN).

In [ ]:
# Định nghĩa hàm chuyển danh sách token thành danh sách chỉ số
def tokens_to_ids(tokens):
    # Sử dụng word_to_idx để ánh xạ token sang chỉ số. Nếu token không có trong từ điển thì dùng chỉ số của <unk> (là 1)
    return [word_to_idx.get(token, 1) for token in tokens]

# Áp dụng hàm tokens_to_ids cho cột 'tokens' trong các DataFrame
train['input_ids'] = train['tokens'].apply(tokens_to_ids)
val['input_ids'] = val['tokens'].apply(tokens_to_ids)
test['input_ids'] = test['tokens'].apply(tokens_to_ids)

# Định nghĩa độ dài cố định cho chuỗi đầu vào sau khi padding/cắt bớt
MAX_LEN = 128
# Định nghĩa hàm padding hoặc cắt bớt chuỗi chỉ số
def pad_sequence(seq):
    # Cắt bớt nếu độ dài lớn hơn MAX_LEN
    # Thêm số 0 (chỉ số của <pad>) vào cuối nếu độ dài nhỏ hơn MAX_LEN
    return seq[:MAX_LEN] + [0] * max(0, MAX_LEN - len(seq))

# Áp dụng hàm pad_sequence cho cột 'input_ids'
train['input_ids'] = train['input_ids'].apply(pad_sequence)
val['input_ids'] = val['input_ids'].apply(pad_sequence)
test['input_ids'] = test['input_ids'].apply(pad_sequence)

#### 6. Chuyển đổi nhãn (label) về dạng số

- Các nhãn gốc của từng khía cạnh (aspect) là: -1, 0, 1, 2.
- Ta ánh xạ các giá trị này về các số 0, 1, 2, 3 để phù hợp với CrossEntropyLoss của PyTorch.
- Thực hiện chuyển đổi cho cả train, val, test.

In [ ]:
# Định nghĩa danh sách các khía cạnh (aspects) cần phân tích
ASPECTS = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others']
# Số lượng lớp (nhãn) cho mỗi khía cạnh (-1, 0, 1, 2)
NUM_CLASSES = 4
# Xác định thiết bị sẽ sử dụng để huấn luyện mô hình (GPU nếu có, ngược lại là CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ánh xạ các nhãn gốc (-1, 0, 1, 2) sang chỉ số (0, 1, 2, 3)
label_map = {-1: 0, 0: 1, 1: 2, 2: 3}

# Lấy nhãn gốc từ cột ASPECTS trong DataFrame train và chuyển sang mảng numpy
train_labels_raw = np.array(train[ASPECTS])
# Áp dụng ánh xạ label_map cho từng phần tử trong mảng nhãn gốc. Nếu có giá trị không mong muốn thì mặc định là 0
train_labels = np.vectorize(lambda x: label_map.get(x, 0))(train_labels_raw)

# Tương tự cho tập test
test_labels_raw = np.array(test[ASPECTS])
test_labels = np.vectorize(lambda x: label_map.get(x, 0))(test_labels_raw)

# Tương tự cho tập validation
val_labels_raw = np.array(val[ASPECTS])
val_labels = np.vectorize(lambda x: label_map.get(x, 0))(val_labels_raw)

#### 7. Định nghĩa Dataset cho PyTorch

- Tạo class kế thừa `torch.utils.data.Dataset` để quản lý dữ liệu đầu vào và nhãn.
- Chuẩn bị DataLoader cho train, val, test.

In [ ]:
# Định nghĩa lớp Dataset tùy chỉnh cho PyTorch
class ABSADataset(Dataset):
    # Hàm khởi tạo, nhận đầu vào (input_ids) và nhãn (labels)
    def __init__(self, inputs, labels):
        # Chuyển input_ids sang tensor PyTorch kiểu long
        self.inputs = torch.tensor(inputs.values.tolist(), dtype=torch.long)
        # Chuyển labels sang tensor PyTorch kiểu long
        self.labels = torch.tensor(labels, dtype=torch.long)

    # Hàm trả về số lượng mẫu trong dataset
    def __len__(self):
        return len(self.inputs)

    # Hàm trả về một mẫu dữ liệu và nhãn tương ứng tại chỉ số idx
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

# Tạo các đối tượng Dataset cho tập train, test, validation
train_data = ABSADataset(train['input_ids'], train_labels)
test_data = ABSADataset(test['input_ids'], test_labels)
val_data = ABSADataset(val['input_ids'], val_labels)

# Tạo DataLoader cho các tập dữ liệu
# DataLoader giúp tải dữ liệu theo batch, xáo trộn dữ liệu (cho tập train)
train_load = DataLoader(train_data, batch_size=32, shuffle=True)
test_load = DataLoader(test_data, batch_size=32) # Không cần xáo trộn tập test
val_load = DataLoader(val_data, batch_size=32) # Không cần xáo trộn tập validation

#### 8. Định nghĩa mô hình LSTM cho ABSA

- Mô hình gồm các lớp: Embedding, LSTM 2 chiều, Dropout, Fully Connected.
- Đầu ra được reshape về (batch_size, num_aspects, num_classes).

In [ ]:
# Định nghĩa mô hình LSTM cho bài toán ABSA
class LSTM_ABSA(nn.Module):
    # Hàm khởi tạo mô hình
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, num_aspects=len(ASPECTS), num_classes=NUM_CLASSES, dropout_prob=0.5):
        super().__init__()
        # Lớp Embedding: chuyển chỉ số từ thành vector mật độ
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) # padding_idx=0 để bỏ qua token <pad>
        # Lớp LSTM hai chiều (bidirectional): xử lý chuỗi
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        # Lớp Dropout: giúp tránh overfitting
        self.dropout = nn.Dropout(dropout_prob)
        # Lớp Fully Connected (Linear): ánh xạ đầu ra của LSTM đến số lượng khía cạnh * số lượng lớp
        self.fc = nn.Linear(hidden_dim * 2, num_aspects * num_classes) # hidden_dim * 2 vì là LSTM 2 chiều
        # Lưu lại số lượng lớp và khía cạnh
        self.num_classes = num_classes
        self.num_aspects = num_aspects

    # Hàm forward: định nghĩa luồng dữ liệu qua mô hình
    def forward(self, x):
        # Ánh xạ đầu vào (chỉ số từ) sang vector embedding
        embedding = self.embedding(x)
        # Truyền embedding qua lớp LSTM
        # _: đầu ra của LSTM cho từng bước thời gian (không dùng ở đây)
        # hn: trạng thái ẩn cuối cùng
        # _: trạng thái cell cuối cùng (không dùng ở đây)
        _, (hn, _) = self.lstm(embedding)
        # Nối trạng thái ẩn cuối cùng của hai chiều (forward và backward)
        hn = torch.cat((hn[0], hn[1]), dim=1)
        # Áp dụng Dropout
        hn = self.dropout(hn)
        # Truyền qua lớp Fully Connected
        out = self.fc(hn)
        # Reshape đầu ra về dạng (batch_size, num_aspects, num_classes)
        return out.view(-1, self.num_aspects, self.num_classes)

# Khởi tạo mô hình LSTM và chuyển lên thiết bị đã chọn (CPU/GPU)
model_lstm = LSTM_ABSA(len(vocab)).to(DEVICE)
# Định nghĩa hàm mất mát (Loss Function): CrossEntropyLoss cho bài toán phân loại đa lớp
criterion = nn.CrossEntropyLoss()
# Định nghĩa thuật toán tối ưu hóa: Adam
optimizer = optim.Adam(model_lstm.parameters(), lr=0.001)

#### 9. Huấn luyện và đánh giá mô hình LSTM

- Huấn luyện mô hình trên tập train, đánh giá trên tập validation.
- Lưu lại trọng số mô hình sau khi huấn luyện.

In [ ]:
# Số lượng epoch (số lần lặp qua toàn bộ tập train)
epoch = 1
# Vòng lặp huấn luyện qua các epoch
for e in range(epoch):
    # Chuyển mô hình sang chế độ huấn luyện
    model_lstm.train()
    total_loss = 0 # Biến lưu tổng mất mát trong epoch
    # Lặp qua từng batch dữ liệu trong train_load
    for input, label in train_load:
        # Chuyển dữ liệu và nhãn lên thiết bị đã chọn
        input, label = input.to(DEVICE), label.to(DEVICE)
        # Đặt gradient về 0 trước khi tính toán
        optimizer.zero_grad()
        # Truyền dữ liệu qua mô hình để nhận đầu ra (dự đoán)
        output = model_lstm(input)
        # Tính tổng mất mát cho tất cả các khía cạnh trong batch hiện tại
        loss = sum(criterion(output[:, i, :], label[:, i]) for i in range(len(ASPECTS)))
        # Lan truyền ngược lỗi để tính gradient
        loss.backward()
        # Cập nhật trọng số của mô hình
        optimizer.step()
        # Cộng dồn mất mát của batch vào tổng mất mát
        total_loss += loss.item()
    # In mất mát trung bình của epoch
    print(f"Epoch {e+1}/{epoch}, Loss: {total_loss/len(train_load)}")

# ĐÁNH GIÁ TRÊN TẬP VALIDATION
# Chuyển mô hình sang chế độ đánh giá (tắt dropout, batch norm...)
model_lstm.eval()
val_pred = [] # Danh sách lưu các dự đoán trên tập validation
val_true = [] # Danh sách lưu các nhãn thật trên tập validation
val_loss = 0 # Biến lưu tổng mất mát trên tập validation
# Tắt tính toán gradient trong quá trình đánh giá để tiết kiệm bộ nhớ và tăng tốc
with torch.no_grad():
    # Lặp qua từng batch dữ liệu trong val_load
    for inputs, labels in val_load:
        # Chuyển dữ liệu và nhãn lên thiết bị đã chọn
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        # Truyền dữ liệu qua mô hình để nhận đầu ra
        outputs = model_lstm(inputs)
        # Cộng dồn mất mát cho tất cả các khía cạnh trong batch hiện tại
        val_loss += sum(criterion(outputs[:, i, :], labels[:, i]).item() for i in range(len(ASPECTS)))
        # Lấy chỉ số của lớp có xác suất cao nhất làm dự đoán và chuyển về numpy
        val_pred.extend(torch.argmax(outputs, dim=2).cpu().numpy())
        # Chuyển nhãn thật về numpy
        val_true.extend(labels.cpu().numpy())
# Tính mất mát trung bình trên tập validation
val_loss /= len(val_load)
# Tính độ chính xác trên tập validation (làm phẳng mảng nhãn và dự đoán trước khi tính)
val_acc = accuracy_score(np.array(val_true).flatten(), np.array(val_pred).flatten())
# In kết quả đánh giá trên tập validation
print(f'Epoch {epoch+1}, Val Loss: {val_loss}, Val Accuracy: {val_acc}')

# Lưu trọng số của mô hình sau khi huấn luyện
torch.save(model_lstm.state_dict(), 'lstm_absa_model.pth')

#### 10. Đánh giá mô hình trên tập test

- Dự đoán và tính độ chính xác trên tập test.

In [ ]:
# Chuyển mô hình sang chế độ đánh giá
model_lstm.eval()
test_preds = [] # Danh sách lưu các dự đoán trên tập test
test_true = [] # Danh sách lưu các nhãn thật trên tập test
# Tắt tính toán gradient
with torch.no_grad():
    # Lặp qua từng batch dữ liệu trong test_load
    for inputs, labels in test_load:
        # Chuyển dữ liệu lên thiết bị đã chọn
        inputs = inputs.to(DEVICE)
        # Truyền dữ liệu qua mô hình để nhận đầu ra
        outputs = model_lstm(inputs)
        # Lấy chỉ số của lớp có xác suất cao nhất làm dự đoán và chuyển về numpy
        test_preds.extend(torch.argmax(outputs, dim=2).cpu().numpy())
        # Chuyển nhãn thật về numpy
        test_true.extend(labels.cpu().numpy())

#### 11. Dự đoán với Spark Pandas UDF (LSTM)

- Sử dụng Spark để dự đoán song song trên tập test với mô hình đã huấn luyện.
- Trả về dự đoán cho từng review dưới dạng chuỗi.
- Đánh giá lại độ chính xác trên Spark DataFrame.

In [ ]:
# Khởi tạo SparkSession
spark = SparkSession.builder.appName('ABSA-PandasUDF').getOrCreate()

# Tạo Spark DataFrame từ cột 'Review' của pandas DataFrame test
spark_test_df = spark.createDataFrame(test[['Review']])

# Định nghĩa hàm dự đoán ABSA sẽ chạy trên từng partition của Spark DataFrame (Pandas UDF)
def predict_absa(reviews):
    # Tải lại mô hình đã huấn luyện (trên CPU để tương thích với Pandas UDF mặc định)
    load_model = LSTM_ABSA(len(vocab)).to('cpu')
    load_model.load_state_dict(torch.load('lstm_absa_model.pth'))
    load_model.eval() # Chuyển sang chế độ đánh giá

    # Tiền xử lý các review trong batch: loại bỏ emoji/ký tự đặc biệt, chuyển chữ thường, tách từ
    tokens = reviews.apply(preprocess)
    # Chuyển token thành chỉ số và padding
    input_ids = tokens.apply(tokens_to_ids).apply(pad_sequence)
    # Chuyển danh sách chỉ số thành tensor PyTorch
    inputs = torch.tensor(input_ids.tolist(), dtype=torch.long)

    # Thực hiện dự đoán với mô hình
    with torch.no_grad():
        outputs = load_model(inputs)
        # Lấy chỉ số lớp có xác suất cao nhất làm dự đoán
        predictions = torch.argmax(outputs, dim=2).numpy()

    # Trả về kết quả dự đoán dưới dạng pandas Series, mỗi phần tử là chuỗi các dự đoán cho các khía cạnh, phân cách bởi dấu ','
    return pd.Series([','.join(map(str, p)) for p in predictions])

# Định nghĩa Pandas UDF sử dụng hàm predict_absa
# returnType='string': kiểu dữ liệu trả về của UDF là chuỗi
# functionType=PandasUDFType.SCALAR: loại UDF là scalar (áp dụng cho từng hàng)
absa_udf = pandas_udf(predict_absa, returnType='string', functionType=PandasUDFType.SCALAR)
# Áp dụng UDF vào Spark DataFrame để tạo cột 'predictions'
result_df = spark_test_df.withColumn('predictions', absa_udf(spark_test_df['Review']))
# Hiển thị kết quả dự đoán (không cắt bớt nội dung)
result_df.show(truncate=False)

# Chuẩn bị dữ liệu để đánh giá độ chính xác bằng Spark ML
# Tạo Spark DataFrame từ kết quả dự đoán và nhãn thật đã có từ quá trình đánh giá bằng PyTorch
preds_df = spark.createDataFrame(pd.DataFrame({'prediction': np.array(test_preds).flatten(), 'label': np.array(test_true).flatten()}))
# Chuyển kiểu dữ liệu của cột 'prediction' và 'label' sang DoubleType để phù hợp với MulticlassClassificationEvaluator
preds_df = preds_df.withColumn('prediction', preds_df['prediction'].cast(DoubleType()))
preds_df = preds_df.withColumn('label', preds_df['label'].cast(DoubleType()))

# Khởi tạo đối tượng đánh giá MulticlassClassificationEvaluator
# labelCol: tên cột chứa nhãn thật
# predictionCol: tên cột chứa dự đoán
# metricName: metric cần tính (độ chính xác)
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
# Tính độ chính xác trên Spark DataFrame
accuracy = evaluator.evaluate(preds_df)
# In độ chính xác trên tập test
print('Test Accuracy:', accuracy)

25/10/07 07:32:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
/usr/local/lib/python3.11/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Review                                                                                                                                                                                                                                               |predictions    |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Giày hơi có mùi nồng, lưu ý đôi LA không phải đế xám nên mng cân nhắc kĩ nhé ạ nhma với giá tiền này thì là oke                                                                                                